In [1]:
import pandas as pd
import numpy as np
import sys
import random
import os
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
sys.path.append('../..')
from modules import utils
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

2023-08-09 13:54:54.680231: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 13:54:54.860021: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-09 13:54:54.888812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-09 13:54:54.888826: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudar

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.random.set_seed(SEED)

#### The data

In [3]:
kampala_devices = pd.read_csv('../data/kampala_devices.csv', usecols=['lat', 'long', 'id'])
kampala_devices.head()

,id,lat,long
0,930434,0.360209,32.610756
1,718028,0.307500,32.620600
2,912224,0.346460,32.703280
3,930426,0.365500,32.646800
4,930427,0.268900,32.588000


In [4]:
kampala_df = pd.read_csv('../data/kampala_data.csv', parse_dates=['timestamp'])
kampala_df.head()

,site_name,latitude,longitude,city,timestamp,pm2_5_calibrated_value,pm2_5_raw_value,pm10_raw_value,pm10_calibrated_value,site_id,device_number,device_name
0,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 00:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
1,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 01:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
2,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 02:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
3,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 03:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26
4,"Civic Centre, Kampala Central",0.317725,32.592509,Kampala,2021-09-01 04:00:00+00:00,NaN,NaN,NaN,NaN,60d058c8048305120d2d6145,689761,aq_26


In [5]:
final_df = pd.DataFrame()
cols = ['timestamp', 'latitude', 'longitude', 'pm2_5_calibrated_value', 'device_number']
# for i, device_id in enumerate(device_ids):
for i, device_id in kampala_devices.id.iteritems():
    device_df = utils.get_device_data(kampala_df, device_id, cols)
    processed_df = utils.preprocessing(device_df)
    final_df = pd.concat([final_df, processed_df])
final_df.reset_index(drop=True, inplace=True)
final_df.head()

,time,latitude,longitude,pm2_5,device_number
0,453031.0,0.356989,32.613888,10.5477,930434
1,453032.0,0.356989,32.613888,16.4250,930434
2,453033.0,0.356989,32.613888,17.7239,930434
3,453034.0,0.356989,32.613888,16.1533,930434
4,453035.0,0.356989,32.613888,18.0123,930434


In [6]:
latitudes = final_df['latitude'].unique()
longitudes = final_df['longitude'].unique()
device_ids = final_df['device_number'].unique()
len(latitudes), len(longitudes), len(device_ids)

(35, 35, 34)

In [7]:
final_df = final_df.drop(['device_number'], axis=1)
final_df.head()

,time,latitude,longitude,pm2_5
0,453031.0,0.356989,32.613888,10.5477
1,453032.0,0.356989,32.613888,16.4250
2,453033.0,0.356989,32.613888,17.7239
3,453034.0,0.356989,32.613888,16.1533
4,453035.0,0.356989,32.613888,18.0123


#### Model training and validation

In [8]:
def lstm(X_train, y_train, epochs=1000, optimizer='RMSProp', dropout=0.2):
#     model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.LSTM(50),
#     tf.keras.layers.Dense(1)
# ])
    
    input_layer = Input(shape=(X_train.shape[1],))
    reshaped_input = tf.expand_dims(input_layer, axis=1)
    
    lstm_layer1 = LSTM(units=50, return_sequences=True)(reshaped_input)
    dropout_layer1 = Dropout(dropout)(lstm_layer1)

    lstm_layer2 = LSTM(units=50)(dropout_layer1)
    dropout_layer2 = Dropout(dropout)(lstm_layer2)
    
    output_layer = Dense(units=1)(dropout_layer2)

    model = Model(inputs=input_layer, outputs=output_layer) 

    model.compile(optimizer=optimizer, loss='mean_squared_error')
    model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_split=0.2)
    
#     model = keras.Sequential()
#     model.add(Input(shape=(X_train.shape[1],), name='Input-Layer')) 
#     model.add(LSTM(50, return_sequences=True, input_shape=(window_size, x_train.shape[-1])))
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM((window_size * 2), return_sequences=True))) 
#     model.add(Dropout(rate=dropout))

#     model.add(Bidirectional(LSTM(window_size, return_sequences=False))) 
#     model.add(Dense(units=1))
    return model

In [9]:
# def scale_data(X):
#     scaler = MinMaxScaler()
#     X_scaled = X.copy()
#     X_scaled[:, 0] = scaler.fit_transform(X[:, 0].reshape(-1, 1)).flatten()
#     return X_scaled

In [10]:
def cross_validation(final_df, idx):
    device_indices = final_df[final_df.latitude==latitudes[idx]].index
    device_df = kampala_df[kampala_df.device_number == device_ids[idx]]
#     assert(len(device_indices) == len(device_df)-device_df.pm2_5_calibrated_value.isna().sum())
    
    test_df = final_df.loc[device_indices]
#     assert(len(test_df.longitude.unique()) == 1)
    
    train_df = pd.concat([final_df, test_df]).drop_duplicates(keep=False)
#     assert(len(train_df.longitude.unique()) == len(longitudes)-1)
#     assert len(final_df) == len(test_df) + len(train_df)
    
    X_train = train_df.iloc[:, 0:-1]
    y_train = train_df.iloc[:, -1]
    X_train, y_train = np.array(X_train), np.array(y_train)#.reshape(-1, 1)
#     X_train_scaled = scale_data(X_train)
    
    X_test = test_df.iloc[:, 0:-1]
    y_test = test_df.iloc[:, -1]
    X_test, y_test = np.array(X_test), np.array(y_test)#.reshape(-1, 1)
#     X_test_scaled = scale_data(X_test)
    
    model = lstm(X_train, y_train) #, dropout=0.1)
    y_pred = model.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    return rmse, mape

In [11]:
len(latitudes)

35

In [ ]:
rmse_list, mape_list = [], []
lat_list = list(latitudes)
for lat in latitudes[8:12]:
    i = lat_list.index(lat)
    try:
        rmse, mape = cross_validation(final_df, i)
        rmse_list.append(rmse)
        mape_list.append(mape)
        print(f'Location {i} successful')
    except Exception as e:
        print(e)
        print(f'Location {i} failed')

mean_rmse = np.mean(rmse_list)
mean_mape = np.mean(mape_list)
mean_rmse, mean_mape

Epoch 1/1000
1164/1164 [==============================] - 10s 6ms/step - loss: 725.5849 - val_loss: 604.9008
Epoch 2/1000
1164/1164 [==============================] - 4s 4ms/step - loss: 483.8851 - val_loss: 604.7911
Epoch 3/1000
1164/1164 [==============================] - 5s 4ms/step - loss: 484.1159 - val_loss: 604.5703
Epoch 4/1000
1164/1164 [==============================] - 7s 6ms/step - loss: 483.0111 - val_loss: 604.0347
Epoch 5/1000
1164/1164 [==============================] - 9s 8ms/step - loss: 483.3082 - val_loss: 604.6223
Epoch 6/1000
1164/1164 [==============================] - 6s 5ms/step - loss: 483.9140 - val_loss: 605.3956
Epoch 7/1000
1164/1164 [==============================] - 5s 4ms/step - loss: 482.7584 - val_loss: 605.6066
Epoch 8/1000
1164/1164 [==============================] - 7s 6ms/step - loss: 483.0674 - val_loss: 604.5074
Epoch 9/1000
1164/1164 [==============================] - 7s 6ms/step - loss: 483.3151 - val_loss: 605.5676
Epoch 10/1000
1164/1164 [==

Epoch 151/1000
1164/1164 [==============================] - 11s 9ms/step - loss: 481.8110 - val_loss: 604.0185
Epoch 152/1000
1164/1164 [==============================] - 12s 11ms/step - loss: 481.6114 - val_loss: 603.7486
Epoch 153/1000
1164/1164 [==============================] - 13s 11ms/step - loss: 481.3574 - val_loss: 603.1279
Epoch 154/1000
1164/1164 [==============================] - 13s 11ms/step - loss: 481.5433 - val_loss: 604.0375
Epoch 155/1000
1164/1164 [==============================] - 13s 11ms/step - loss: 481.7774 - val_loss: 603.7096
Epoch 156/1000
1164/1164 [==============================] - 13s 11ms/step - loss: 481.8129 - val_loss: 603.9346
Epoch 157/1000
1164/1164 [==============================] - 12s 11ms/step - loss: 481.4276 - val_loss: 603.0559
Epoch 158/1000
1164/1164 [==============================] - 12s 11ms/step - loss: 481.2249 - val_loss: 603.3005
Epoch 159/1000
1164/1164 [==============================] - 13s 11ms/step - loss: 481.2371 - val_loss: 60

1164/1164 [==============================] - 3s 3ms/step - loss: 480.5556 - val_loss: 602.9066
Epoch 226/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.0020 - val_loss: 602.9046
Epoch 227/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.0928 - val_loss: 602.9953
Epoch 228/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.9714 - val_loss: 602.9247
Epoch 229/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 481.3774 - val_loss: 602.9596
Epoch 230/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.4672 - val_loss: 602.9094
Epoch 231/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.8067 - val_loss: 602.9240
Epoch 232/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.4546 - val_loss: 602.9051
Epoch 233/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.5911 - val_loss: 603.0734
Epoch 234/1000
1164/1164 

1164/1164 [==============================] - 3s 3ms/step - loss: 479.6449 - val_loss: 602.9667
Epoch 374/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.0191 - val_loss: 603.0831
Epoch 375/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.3865 - val_loss: 602.9069
Epoch 376/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.1133 - val_loss: 602.9111
Epoch 377/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.6154 - val_loss: 603.2516
Epoch 378/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.5207 - val_loss: 603.0258
Epoch 379/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.0870 - val_loss: 602.9062
Epoch 380/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.3480 - val_loss: 602.9225
Epoch 381/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 480.3627 - val_loss: 603.0619
Epoch 382/1000
1164/1164 

1164/1164 [==============================] - 3s 3ms/step - loss: 479.4204 - val_loss: 603.3964
Epoch 522/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.4344 - val_loss: 603.0777
Epoch 523/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.3574 - val_loss: 603.6016
Epoch 524/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.3715 - val_loss: 603.1924
Epoch 525/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.3469 - val_loss: 603.2892
Epoch 526/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.2383 - val_loss: 603.2150
Epoch 527/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.4400 - val_loss: 603.0783
Epoch 528/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.2169 - val_loss: 602.9297
Epoch 529/1000
1164/1164 [==============================] - 3s 3ms/step - loss: 479.2313 - val_loss: 602.9258
Epoch 530/1000
1164/1164 

...

1167/1167 [==============================] - 4s 3ms/step - loss: 529.9651 - val_loss: 602.5764
Epoch 189/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 530.0759 - val_loss: 601.4899
Epoch 190/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.6816 - val_loss: 601.6617
Epoch 191/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 530.4294 - val_loss: 601.5394
Epoch 192/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 530.0378 - val_loss: 601.4948
Epoch 193/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.7253 - val_loss: 602.4753
Epoch 194/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.3561 - val_loss: 602.0754
Epoch 195/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.2379 - val_loss: 602.2888
Epoch 196/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.6920 - val_loss: 601.7093
Epoch 197/1000
1167/1167 

1167/1167 [==============================] - 4s 3ms/step - loss: 528.6954 - val_loss: 601.8391
Epoch 337/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.0055 - val_loss: 602.9722
Epoch 338/1000
1167/1167 [==============================] - 3s 3ms/step - loss: 529.3769 - val_loss: 602.6743
Epoch 339/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.5938 - val_loss: 602.0151
Epoch 340/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.7737 - val_loss: 604.5079
Epoch 341/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.6006 - val_loss: 603.0333
Epoch 342/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.7925 - val_loss: 602.7074
Epoch 343/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 529.1802 - val_loss: 603.5346
Epoch 344/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.3243 - val_loss: 603.2021
Epoch 345/1000
1167/1167 

1167/1167 [==============================] - 4s 3ms/step - loss: 528.4304 - val_loss: 602.1588
Epoch 485/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.7329 - val_loss: 602.8700
Epoch 486/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.9009 - val_loss: 602.9967
Epoch 487/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.0961 - val_loss: 603.5650
Epoch 488/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.0925 - val_loss: 602.1814
Epoch 489/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.8824 - val_loss: 601.4849
Epoch 490/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.8908 - val_loss: 601.5182
Epoch 491/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.7027 - val_loss: 603.1899
Epoch 492/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 528.0741 - val_loss: 602.2721
Epoch 493/1000
1167/1167 

1167/1167 [==============================] - 4s 3ms/step - loss: 527.0408 - val_loss: 601.5675
Epoch 633/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.6815 - val_loss: 601.5297
Epoch 634/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.2762 - val_loss: 601.5448
Epoch 635/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.4279 - val_loss: 601.4857
Epoch 636/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.5767 - val_loss: 602.0538
Epoch 637/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.2320 - val_loss: 601.9811
Epoch 638/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.2529 - val_loss: 601.7005
Epoch 639/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.4497 - val_loss: 601.4897
Epoch 640/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 527.4122 - val_loss: 601.5566
Epoch 641/1000
1167/1167 

1167/1167 [==============================] - 4s 3ms/step - loss: 526.8308 - val_loss: 601.8139
Epoch 781/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.8293 - val_loss: 601.7357
Epoch 782/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.9766 - val_loss: 601.8997
Epoch 783/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.8293 - val_loss: 602.1238
Epoch 784/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.8699 - val_loss: 601.7708
Epoch 785/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.8988 - val_loss: 601.8636
Epoch 786/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.9091 - val_loss: 601.6243
Epoch 787/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.9052 - val_loss: 601.7007
Epoch 788/1000
1167/1167 [==============================] - 4s 3ms/step - loss: 526.8444 - val_loss: 601.8244
Epoch 789/1000
1167/1167 

In [1]:
rmse_list

NameError: name 'rmse_list' is not defined

In [ ]:
mape_list